<a href="https://colab.research.google.com/github/balszeg/Thesis/blob/main/Faster_R_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Utilities

Important to set the Colab runtime type to **GPU**.

In [ ]:
# clone TensorFlow models repository
!git clone https://github.com/tensorflow/models.git

In [ ]:
# installing the necessary libraries, packages
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

In [ ]:
# with the newest TensorFlow 2.5 the model cannot utilize GPU
# hence it is recommended to downgrade it to the last stable version
!pip install tensorflow==2.4.1

In [ ]:
# this script check if everything was installed correctly
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

## Data preprocessing

In [ ]:
# create directories for the data
!mkdir /content/dataset
!mkdir /content/dataset/train/
!mkdir /content/dataset/train/annos/
!mkdir /content/dataset/val/
!mkdir /content/dataset/val/annos/

In [ ]:
# copy the original BDD100K dataset from drive
!cp /content/gdrive/MyDrive/bdd100k/bdd100k_images.zip /content/dataset/
!cp /content/gdrive/MyDrive/bdd100k/bdd100k_labels_release.zip /content/dataset/

In [ ]:
# unzip it
%cd /content/dataset
!unzip bdd100k_images.zip
!unzip bdd100k_labels_release.zip

In [ ]:
# clone Monk Object Detection repository for .tfrecord converter
%cd /content/
!git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git

In [ ]:
# the conversion order is the following: 
# BDD100K(.jpg, .txt)->Pascal VOC(.jpg, .xml)->TensorFlow (.record/.tfrecord)
# install Pascal VOC writer
!pip install pascal-voc-writer

In [ ]:
# set directories for training set
img_dir = "/content/dataset/bdd100k/images/100k/train/"
anno_file = "/content/dataset/bdd100k/labels/bdd100k_labels_images_train.json"
output_dir = "/content/dataset/train/annos"

In [ ]:
# parse through the training label .json
import json
with open(anno_file) as json_file:
    data = json.load(json_file)

len(data)

In [ ]:
# convert it to .xml files
from pascal_voc_writer import Writer

import os
from tqdm import tqdm
import cv2

for i in tqdm(range(len(data))):
    name = data[i]["name"];
    labels = data[i]["labels"];
    #print(img_dir + "/" + name)
    img = cv2.imread(img_dir + "/" + name);
    h, w, c = img.shape;
    writer = Writer(name, w, h)
    for j in range(len(labels)):
        obj = labels[j];
        if("box2d" in obj.keys()):
            category = obj["category"];
            x1 = int(obj["box2d"]["x1"]);
            y1 = int(obj["box2d"]["y1"]);
            x2 = int(obj["box2d"]["x2"]);
            y2 = int(obj["box2d"]["y2"]);
            writer.addObject(category, x1, y1, x2, y2);
    
    writer.save(output_dir + "/" + name.split(".")[0] + ".xml")

        
    #break;

In [ ]:
# creat labelmap.txt with classes
import os
from tqdm import tqdm
import cv2
classes = [];

for i in tqdm(range(len(data))):
    name = data[i]["name"];
    labels = data[i]["labels"];
    for j in range(len(labels)):
        obj = labels[j];
        if("box2d" in obj.keys()):
            category = obj["category"];
            if(category not in classes):
                classes.append(category)

classes

In [ ]:
f = open("classes.txt", 'w');
for i in range(len(classes)):
    f.write(classes[i] + "\n");
f.close();

In [ ]:
# set directories for validation set
img_dir = "/content/dataset/bdd100k/images/100k/val";
anno_file = "/content/dataset/bdd100k/labels/bdd100k_labels_images_val.json";
output_dir = "/content/dataset/val/annos";

In [ ]:
# parse through the validation label .json
import json
with open(anno_file) as json_file:
    data = json.load(json_file)

len(data)

In [ ]:
# convert it to .xml files
from pascal_voc_writer import Writer

import os
from tqdm import tqdm
import cv2

for i in tqdm(range(len(data))):
    name = data[i]["name"];
    labels = data[i]["labels"];
    #print(img_dir + "/" + name)
    img = cv2.imread(img_dir + "/" + name);
    h, w, c = img.shape;
    writer = Writer(name, w, h)
    for j in range(len(labels)):
        obj = labels[j];
        if("box2d" in obj.keys()):
            category = obj["category"];
            x1 = int(obj["box2d"]["x1"]);
            y1 = int(obj["box2d"]["y1"]);
            x2 = int(obj["box2d"]["x2"]);
            y2 = int(obj["box2d"]["y2"]);
            writer.addObject(category, x1, y1, x2, y2);
    
    writer.save(output_dir + "/" + name.split(".")[0] + ".xml")

        
    #break;

In [ ]:
# import libraries for .record creations
import os
import sys
sys.path.append("Monk_Object_Detection/13_tf_obj_2/lib/")

from train_detector import Detector
gtf = Detector();

In [ ]:
# set up the path for the training data
train_img_dir = "/content/dataset/bdd100k/images/100k/train/";
train_anno_dir = "/content/dataset/train/annos";
class_list_file = "classes.txt";

gtf.set_train_dataset(train_img_dir, train_anno_dir, class_list_file, batch_size=4)

In [ ]:
# same set up with validation set
val_img_dir = "/content/dataset/bdd100k/images/100k/val/";
val_anno_dir = "/content/dataset/val/annos";

gtf.set_val_dataset(val_img_dir, val_anno_dir)

In [ ]:
# creat both set in .record form
gtf.create_tfrecord(data_output_dir="data_tfrecord")

## TensorBoard setup

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=/content/models/research/object_detection/training/train/

If this won't open it for some reason, there is an alternative way.

In [ ]:
%cd /content/
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

In [ ]:
LOG_DIR = "/content/models/research/object_detection/training/train/"
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

## Training

In [ ]:
# download the official config, if training for the first time
%cd /content/models/research/object_detection
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
!tar -xvf faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz

In [ ]:
%cd /content/models/research/object_detection
!mkdir training
!cp /content/models/research/object_detection/configs/tf2/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config /content/models/research/object_detection/training

In [ ]:
!python model_main_tf2.py --pipeline_config_path=training/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config --model_dir=training --alsologtostderr

## Evaluation

In [ ]:
# export the trained model for later inference
!python exporter_main_v2.py --trained_checkpoint_dir training --output_directory inference_graph --pipeline_config_path training/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config

In [ ]:
# calculate the metrics
!python model_main_tf2.py --pipeline_config_path=training/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config --model_dir=training --checkpoint_dir=training --alsologtostderr

In [ ]:
# for video inference, an another git repository needed
%cd /content
!git clone https://github.com/alen-smajic/Real-time-Object-Detection-for-Autonomous-Driving-using-Deep-Learning

In [ ]:
# testing on a video
%cd /content/Real-time-Object-Detection-for-Autonomous-Driving-using-Deep-Learning/Faster R-CNN
!python3 detect_objects.py --model_path /content/inference_graph/saved_model --path_to_labelmap /content/vodeo/labelmap.txt --video_input --video_path /content/vodeo/selfmade.mp4 --save_output